In [1]:
import re
import numpy as np
import torch
import torch.nn as nn
from torchmetrics.functional import accuracy
import torchmetrics
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data preprocessing

In [2]:

def load_reviews(path, review_length, vocab):
    reviews = []
    f = open(path, "r", encoding='UTF-8')
    
    for line in f:
        review = []
        line = re.sub(r'[^\w\s]', '', line).lower().strip()
        for word in line.split():
            review.append(vocab[word])
        if len(review) < review_length: #pad in front
            for i in range(review_length - len(review)):
                review.insert(0, "0")
        else: # truncate
            review = review[:review_length]
        reviews.append(review)
    return np.array(reviews)


def find_vocab(path):
    f = open(path, "r", encoding='UTF-8')
    vocab = set()
    for line in f:
        line = re.sub(r'[^\w\s]', '', line).lower().strip() # reference to geeksforgeeks
        for word in line.split():
            vocab.add(word)  
    return {word: i for i, word in enumerate(vocab, 1)} # enumerate start at 1 to leave 0 for padding
        

def prepare_train(data_path, review_length=400):
    all_vocab = find_vocab(data_path + "all_merged.txt")
    train_positive_reviews = load_reviews(data_path + "train_pos_merged.txt", review_length, all_vocab).astype(np.int32)
    train_negative_reviews = load_reviews(data_path + "train_neg_merged.txt", review_length, all_vocab).astype(np.int32)
    test_positive_reviews = load_reviews(data_path + "test_pos_merged.txt", review_length, all_vocab).astype(np.int32)
    test_negative_reviews = load_reviews(data_path + "test_neg_merged.txt", review_length, all_vocab).astype(np.int32)
    
    train_x = np.concatenate((train_positive_reviews, train_negative_reviews), axis=0)
    train_y = np.concatenate(((np.full((len(train_positive_reviews), 1), 1)), (np.full((len(train_negative_reviews), 1), 0))), axis=0)
    test_x = np.concatenate((test_positive_reviews, test_negative_reviews), axis=0)
    test_y = np.concatenate(((np.full((len(test_positive_reviews), 1), 1)), (np.full((len(test_negative_reviews), 1), 0))), axis=0)                         
    print("Vocabulary size: ", len(all_vocab))
    # print(train_positive_reviews[:100])
    #print(train_x.shape)
    #print(train_y.shape)
    #print(test_x.shape)
    #print(test_y.shape)
    return train_x, train_y, test_x, test_y, len(all_vocab)

# train_x, train_y, test_x, test_y = prepare_train(data_path)

## Build RNN with GRU

In [3]:
class MyGRU(nn.Module):
    def __init__(self, vocab_size, hidden_dim, output_dim, num_layers, batch_size=512): # TODO: add dropout
        super().__init__()
        self.last_hidden = torch.zeros(num_layers, batch_size, hidden_dim).to(device)
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.gru = nn.GRU(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        # self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.embedding(x)
        output, hidden = self.gru(x, self.last_hidden)
        output = output[:, -1]
        self.last_hidden = hidden.data # TODO: .data?
        #print(self.last_hidden.shape)
        
        return self.fc(output)

In [4]:
def train_test_GRU(train_loader, test_loader, learning_rate, epochs, vocab_size, hidden_dim, batch_size):
    model = MyGRU(vocab_size=vocab_size, hidden_dim=hidden_dim, output_dim=1, num_layers=2, batch_size=batch_size)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    model.to(device)
    train_loss_all = []
    test_acc_all = []
    
    for epoch in range(1, epochs+1):
        #train
        train_loss = 0
        #train_acc = 0
        model.train()
        for batch in tqdm(train_loader):
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = loss_fn(output, y.float())# TODO: squeeze?
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            #train_acc += accuracy(output, y, task='binary')
        print(f"Epoch: {epoch}, Train Loss: {train_loss / len(train_loader)}")#, Train Acc: {train_acc / len(train_loader)
    
        #test
        #test_loss = 0
        test_acc = 0
        for batch in tqdm(test_loader):
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            output = torch.sigmoid(model(x)) # TODO: squeeze?
            loss = loss_fn(output, y.float())

            #test_loss += loss.item()
            test_acc += accuracy(output, y, task='binary')
        print(f"Epoch: {epoch}, Test Acc: {test_acc / len(test_loader)}") #Test Loss: {test_loss / len(test_loader)},

        train_loss_all.append(train_loss / len(train_loader))
        test_acc_all.append(test_acc / len(test_loader))
        
    return train_loss_all, test_acc_all
    

In [7]:
def main():
    data_path = "./data/"
    review_length = 400
    batch_size = 128
    train_x, train_y, test_x, test_y, vocab_size = prepare_train(data_path, review_length)
    
    train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
    test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
    test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True)
    
    lr = 0.0001
    epochs = 20
    hidden_dim = 256
    
    
    train_loss_all, test_acc_all = train_test_GRU(train_loader, test_loader, learning_rate=lr, epochs=epochs, vocab_size=vocab_size, hidden_dim=hidden_dim, batch_size=batch_size)

main()

Vocabulary size:  53992


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 11.96it/s]


Epoch: 1, Train Loss: 0.6908808506053427


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 31.63it/s]


Epoch: 1, Test Acc: 0.5604619383811951


...

Epoch: 19, Test Acc: 0.7350543737411499


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 11.53it/s]


Epoch: 20, Train Loss: 0.06337835412958394


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 30.87it/s]

Epoch: 20, Test Acc: 0.7432065606117249


In [8]:
## Comparison with a MLP

In [9]:
class mlp(nn.Module):
    def __init__(self):
        super().__init__()
        self.first = nn.Linear(400, 64)
        self.second = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = x.to(torch.float32)
        x = self.first(x)
        x = self.sigmoid(x)
        x = self.second(x)
        x = self.sigmoid(x)
        return x
    
def train_test_mlp(train_loader, test_loader, epochs):
    model = mlp()
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters())
    
    model.to(device)
    train_loss_all = []
    test_acc_all = []
    
    print("Now training MLP model...")
    
    for epoch in range(1, epochs+1):
        #train
        train_loss = 0
        #train_acc = 0
        model.train(True)
        for batch in tqdm(train_loader):
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = loss_fn(output, y.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            #train_acc += accuracy(output, y, task='binary')
        print(f"Epoch: {epoch}, Train Loss: {loss}")#, Train Acc: {train_acc / len(train_loader)
    
        #test
        #test_loss = 0
        test_acc = 0
        model.train(False)
        for batch in tqdm(test_loader):
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            # loss = loss_fn(output, y.float())

            #test_loss += loss.item()
            test_acc += accuracy(output, y, task='binary')
        print(f"Epoch: {epoch}, Test Acc: {test_acc / len(test_loader)}") #Test Loss: {test_loss / len(test_loader)},

        train_loss_all.append(train_loss / len(train_loader))
        test_acc_all.append(test_acc / len(test_loader))
        
    return train_loss_all, test_acc_all

def mlp_main():
    data_path = "./data/"
    review_length = 400
    batch_size = 128
    train_x, train_y, test_x, test_y, vocab_size = prepare_train(data_path, review_length)
    
    train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
    test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
    test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True)
    
    epochs = 20
    
    train_loss_all, test_acc_all = train_test_mlp(train_loader, test_loader, epochs=epochs)
    
mlp_main()

Vocabulary size:  53992
Now training MLP model...


100%|█████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 354.93it/s]


Epoch: 1, Train Loss: 0.6913754940032959


100%|█████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 378.05it/s]


Epoch: 1, Test Acc: 0.4989809989929199


...

Epoch: 19, Test Acc: 0.49796196818351746


100%|█████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 349.42it/s]


Epoch: 20, Train Loss: 0.6930109858512878


100%|█████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 435.13it/s]

Epoch: 20, Test Acc: 0.49966034293174744
